In [122]:
import pandas as pd
import itertools



In [32]:
race_table = pd.read_pickle("data/race_table_clean.pickle")
race_placements = pd.read_pickle("data/race_placements_clean.pickle")

In [33]:
race_placements['pla'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       'WV', '13', '14', 'WX-A', 'WV-A', 'FE', 'WX', 'UR', 'PU', 'DISQ',
       'TNP', 'DNF', ''], dtype=object)

In [31]:
#race_placements['pla'] = race_placements['pla'].str.replace(r"(\d+) DH", r"\1")

In [45]:
race_placements.columns



Index(['pla', 'horse_no', 'horse', 'jockey', 'trainer', 'actual_wt',
       'declare_horse_wt', 'draw', 'LBW', 'finish_time', 'win_odds',
       'race_number', 'race_date', 'race_track', 'update_datetime', 'horse_id',
       'running_position1', 'running_position2', 'running_position3',
       'running_position4'],
      dtype='object')

In [52]:
race_table.head()

,race_number,date,track,Going,Winnings,win_combination,win_dividend,update_datetime,race_class,race_distance,turf,all_weather,course_type
0,247,2018-12-12,Happy Valley,GOOD,920000.0,3,37.5,2019-01-04 09:44:39.269470,Class 4,1200,True,False,C
1,248,2018-12-12,Happy Valley,GOOD,690000.0,5,484.5,2019-01-04 09:44:51.571108,Class 5,1650,True,False,C
2,249,2018-12-12,Happy Valley,GOOD,920000.0,10,32.0,2019-01-04 09:45:04.184324,Class 4,1200,True,False,C
3,250,2018-12-12,Happy Valley,GOOD,920000.0,8,53.5,2019-01-04 09:45:16.618422,Class 4,1800,True,False,C
4,251,2018-12-12,Happy Valley,GOOD,1380000.0,9,135.0,2019-01-04 09:45:29.234692,Class 3,1200,True,False,C


In [86]:
#race_placements.loc[:, "race_date"].isna().sum()
#race_table.loc[:, "date"].isna().sum()

race_placements.shape

(57621, 20)

In [87]:

merged_placements = race_placements.merge(race_table, how="left", left_on=["race_date", "race_number"], right_on=["date", "race_number"])





In [114]:

#merged_placements.loc[:, ["date", "race_number", "race_date"]]
#merged_placements["date"]

#race_placements.loc[:, "race_date"].isna().sum()
#race_table.loc[:, "date"].isna().sum()

merged_placements = merged_placements.dropna()



In [46]:

def past_perf(races_df, by):
    # filter out future dates too
    #place_table = races_df[races_df["date"] > datetime.datetime.now() + pd.Timedelta("-180 days")]
    def past(races, by, bid, when):
        #display(when)
        #past_races = races.copy()
        #past_races = past_races[past_races["date"] < when]
        past_races = races.query("race_date < '{}' and {} == '{}'".format(when, by, bid)) #.strftime('%Y-%m-%d'), by, bid))
        #display(past_races)
        past_races2 = pd.DataFrame()
        #if len(past_races) > 3:
        #    #isplay(past_races["pla"])
        #    past_races2 = past_races.groupby(by)["pla"]\
        #        .value_counts()\
        #        .unstack(fill_value=0)
        #    #display(past_races)
        #    past_races2 = past_races2.apply(lambda x: x/sum(x), axis=1)
        #    #display(past_races)
        #display(past_races)
        for i in ['12', '7', '6', '8', '2', '3', '5', '4', '1', '9', '11', '14', '10', '13', 'WV', 'WV-A', 'WX-A', 'UR', 'WX', 'DISQ', 'FE']:
            #display(past_races.loc[:, "pla"].tolist().count(i))
            past_races2.loc[0, i] = past_races.loc[:, "pla"].tolist().count(i)
            if i not in past_races2.columns:
                past_races2.loc[0, i] = 0
            #else:
                #past_races2.loc[0, i] = past_races.loc[:, "pla"].tolist().count(i)
                #display(past_races2)
        #display(past_races2)
        past_races2 = past_races2.apply(lambda x: x/sum(x) , axis=1)
        past_races2 = past_races2.fillna(0)
        past_races2["place_bet"] = past_races2["1"] + past_races2["2"] + past_races2["3"]
        past_races2.columns = ["past_" + by + "_" + c for c in past_races2.columns]
        #display(past_races)
        #display(past_races2.iloc[0, :])
        #display(past_races2.tolist())
        return past_races2.iloc[0,:]

    #place_table = races_df.copy()
    #place_table = place_table.groupby(by)["pla"].value_counts().unstack(fill_value=0)
    #place_table = place_table.apply(lambda x: x/sum(x), axis=1)
    #place_table["place_bet"] = place_table["01"] + place_table["02"] + place_table["03"]
    #place_table.columns = ["past_" + by + "_" + c for c in place_table.columns]
    #display(past(races_df, by, races_df.loc[5, "date"]))
    
    #return past(races_df, by, races_df.loc[50, by], races_df.loc[50, "date"])
    res = races_df.iloc[:2000, :].apply(lambda x: past(races_df, by, x[by] ,x["race_date"]), axis=1, result_type="expand")
    #display(res)
    res2 = races_df.loc[:2000, :].merge(res, how="left", left_index=True, right_index=True)
    res2[res.columns] = res2[res.columns].fillna(0)
    #display(res2)
    return res2
    
    #display(races[:10, :].apply(lambda x: ))
    
    #race_df = races_df.iloc[:50, :].apply(lambda x: past(races_df, by, x["date"]), axis=1)
    #display(race_df)
    
    #race_df = races_df.copy()
    #race_df = race_df.merge(place_table, how="left", left_on=by, right_on=by)
    #display(race_df)
    #race_df[place_table.columns] = race_df[place_table.columns].fillna(0)
    
    #return race_df
            
        

#display(races.loc[50, "date"].strftime('%Y-%m-%d'))
#display(races.head())        

#%timeit -n 1 -r 1 display(past_perf(races, "horse_id")) # ["horse_id", "jockey_id", "trainer_id"]

display(past_perf(race_placements, "jockey"))

,pla,horse_no,horse,jockey,trainer,actual_wt,declare_horse_wt,draw,LBW,finish_time,...,past_jockey_10,past_jockey_13,past_jockey_WV,past_jockey_WV-A,past_jockey_WX-A,past_jockey_UR,past_jockey_WX,past_jockey_DISQ,past_jockey_FE,past_jockey_place_bet
0,1,3.0,WALDORF,K Teetan,A S Cruz,129.0,1092.0,12.0,0.00,6994.0,...,0.069019,0.022877,0.015898,0.002714,0.000775,0.000775,0.001551,0.000000,0.000388,0.249709
1,2,1.0,NOBLE DELIGHT,J Moreira,J Size,133.0,1016.0,4.0,1.00,7014.0,...,0.033839,0.010891,0.010891,0.003501,0.000389,0.000778,0.000000,0.000778,0.000000,0.500583
2,3,4.0,TRAVEL DATUK,M Chadwick,R Gibson,127.0,1110.0,10.0,2.00,7033.0,...,0.089760,0.030065,0.018736,0.003050,0.000436,0.000000,0.000000,0.000000,0.000436,0.209150
3,4,10.0,SPRING WIN,N Callan,P F Yiu,122.0,1076.0,2.0,2.00,7040.0,...,0.070244,0.023560,0.014398,0.001745,0.000000,0.000436,0.000436,0.000000,0.000873,0.250873
4,5,8.0,HAPPY COOPERATION,C Murray,T P Yung,124.0,1182.0,3.0,4.00,7068.0,...,0.055901,0.037267,0.031056,0.000000,0.006211,0.000000,0.000000,0.000000,0.000000,0.068323
5,6,9.0,LONDON MASTER,U Rispoli,D E Ferraris,123.0,1079.0,7.0,4.00,7070.0,...,0.064669,0.023659,0.013407,0.003155,0.000000,0.001577,0.000789,0.000000,0.000000,0.228707
6,7,2.0,VERY RICH MAN,M F Poon,L Ho,130.0,1077.0,1.0,5.00,7073.0,...,0.069257,0.013514,0.018581,0.005068,0.000000,0.000000,0.000000,0.000000,0.000000,0.293919
7,8,11.0,FIRE AND GOLD,A Badel,C H Yip,118.0,1071.0,9.0,5.00,7075.0,...,0.080925,0.020231,0.017341,0.002890,0.000000,0.000000,0.000000,0.000000,0.000000,0.236994
8,9,5.0,JUNZI,S de Sousa,W Y So,125.0,1015.0,8.0,5.00,7079.0,...,0.101737,0.007444,0.012407,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.287841
9,10,12.0,OPEN HOUSE,C Y Ho,D J Hall,115.0,1025.0,6.0,5.00,7084.0,...,0.082778,0.039623,0.016869,0.001962,0.000392,0.000785,0.000785,0.000000,0.000785,0.171440


In [119]:
def best_perf(races):
    
    def past(races, when, by, bid, dist):
        #past_races = races.copy()
        past_race = races.query("race_date < '{}' and {} == '{}' and race_distance == '{}'".format(when, by, bid, dist))
        if len(past_race) >= 3:
            return past_race.finish_time.min() 
        else:
            return 0
        
    #past(races, races.loc[50,"date"], "horse_id", races.loc[50, "horse_id"], races.loc[50, "dist"])
    races2 = races.copy()
    races2.loc[:, 'max_perf'] = races.apply(lambda x: past(races, x['race_date'], "horse_id", x['horse_id'], x['race_distance']), axis=1, result_type="expand")
    races2.loc[:, "max_perf"] = races2.loc[:, ["finish_time", "max_perf"]].apply(lambda x: x["finish_time"] if x["max_perf"] == 0 else x["max_perf"], axis = 1)
    
    #display(races.merge(races.groupby('horse_id')['dist_per_sec'].max(), how="left", on="horse_id"))
    return races2



best_perf(merged_placements.loc[:3000, :])







,pla,horse_no,horse,jockey,trainer,actual_wt,declare_horse_wt,draw,LBW,finish_time,...,Winnings,win_combination,win_dividend,update_datetime_y,race_class,race_distance,turf,all_weather,course_type,max_perf
0,1,3.0,WALDORF,K Teetan,A S Cruz,129.0,1092.0,12.0,0.00,6994.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,6994.0
1,2,1.0,NOBLE DELIGHT,J Moreira,J Size,133.0,1016.0,4.0,1.00,7014.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7014.0
2,3,4.0,TRAVEL DATUK,M Chadwick,R Gibson,127.0,1110.0,10.0,2.00,7033.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7033.0
3,4,10.0,SPRING WIN,N Callan,P F Yiu,122.0,1076.0,2.0,2.00,7040.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7040.0
4,5,8.0,HAPPY COOPERATION,C Murray,T P Yung,124.0,1182.0,3.0,4.00,7068.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7068.0
5,6,9.0,LONDON MASTER,U Rispoli,D E Ferraris,123.0,1079.0,7.0,4.00,7070.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7120.0
6,7,2.0,VERY RICH MAN,M F Poon,L Ho,130.0,1077.0,1.0,5.00,7073.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7073.0
7,8,11.0,FIRE AND GOLD,A Badel,C H Yip,118.0,1071.0,9.0,5.00,7075.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7075.0
8,9,5.0,JUNZI,S de Sousa,W Y So,125.0,1015.0,8.0,5.00,7079.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7079.0
9,10,12.0,OPEN HOUSE,C Y Ho,D J Hall,115.0,1025.0,6.0,5.00,7084.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,7084.0


In [116]:
def jockey_change(races):
    def past_jockey(races_df, date, hid, cur_jockey):
        #races_df2 = races_df.copy()
        races_df2 = races_df.query("date < '{}' and horse_id == '{}'".format(date, hid)).sort_values("date", ascending=False)['jockey']
        #if len(races_df2.tolist()) >= 1:
            #print(cur_jockey, races_df2.tolist()[0])
        if len(races_df2.tolist()) > 1 and cur_jockey != races_df2.tolist()[0]:
            return 1
        else:
            return 0
    #races.query("")
    
    #display(races.apply(lambda x: past_jockey(races, x['date'], x['horse_id'], x['jockey'] ), axis=1, result_type="expand" ))
    races2 = races.copy()
    races2["jockey_change"] = races.apply(lambda x: past_jockey(races, x['race_date'], x['horse_id'], x['jockey'] ), axis=1, result_type="expand" )
    #display(races.set_index("date").rolling(window=2).mean())
    return races2
    
jockey_change(merged_placements)

,pla,horse_no,horse,jockey,trainer,actual_wt,declare_horse_wt,draw,LBW,finish_time,...,Winnings,win_combination,win_dividend,update_datetime_y,race_class,race_distance,turf,all_weather,course_type,jockey_change
0,1,3.0,WALDORF,K Teetan,A S Cruz,129.0,1092.0,12.0,0.00,6994.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
1,2,1.0,NOBLE DELIGHT,J Moreira,J Size,133.0,1016.0,4.0,1.00,7014.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
2,3,4.0,TRAVEL DATUK,M Chadwick,R Gibson,127.0,1110.0,10.0,2.00,7033.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
3,4,10.0,SPRING WIN,N Callan,P F Yiu,122.0,1076.0,2.0,2.00,7040.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
4,5,8.0,HAPPY COOPERATION,C Murray,T P Yung,124.0,1182.0,3.0,4.00,7068.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
5,6,9.0,LONDON MASTER,U Rispoli,D E Ferraris,123.0,1079.0,7.0,4.00,7070.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
6,7,2.0,VERY RICH MAN,M F Poon,L Ho,130.0,1077.0,1.0,5.00,7073.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
7,8,11.0,FIRE AND GOLD,A Badel,C H Yip,118.0,1071.0,9.0,5.00,7075.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
8,9,5.0,JUNZI,S de Sousa,W Y So,125.0,1015.0,8.0,5.00,7079.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,1
9,10,12.0,OPEN HOUSE,C Y Ho,D J Hall,115.0,1025.0,6.0,5.00,7084.0,...,920000.0,3.0,37.5,2019-01-04 09:44:39.269470,Class 4,1200.0,True,False,C,0


In [133]:



def distance_place(races):
    #distance_place()
    def past_dist_place(race_df):
        race_df.query("date < {}".format())
        
    
    
    new_races = races.copy()
    df = races.groupby(["horse_id", "race_distance"])["pla"].value_counts().unstack(fill_value=0).unstack(fill_value=0)[["1", "2", "3"]]
    df.columns = ["dist_" + str(k[1]) + "_" + k[0] for k in itertools.product(*df.columns.levels) if k[0] in ["1", "2", "3"]]
    
    new_races = new_races.merge(df, how="left", left_on="horse_id", right_index=True)
    return new_races
    #df 

distance_place(merged_placements)

,pla,horse_no,horse,jockey,trainer,actual_wt,declare_horse_wt,draw,LBW,finish_time,...,dist_2400.0_2,dist_1000.0_3,dist_1200.0_3,dist_1400.0_3,dist_1600.0_3,dist_1650.0_3,dist_1800.0_3,dist_2000.0_3,dist_2200.0_3,dist_2400.0_3
0,1,3.0,WALDORF,K Teetan,A S Cruz,129.0,1092.0,12.0,0.00,6994.0,...,0,0,2,0,0,0,0,0,0,0
1,2,1.0,NOBLE DELIGHT,J Moreira,J Size,133.0,1016.0,4.0,1.00,7014.0,...,0,0,0,0,0,0,0,0,0,0
2,3,4.0,TRAVEL DATUK,M Chadwick,R Gibson,127.0,1110.0,10.0,2.00,7033.0,...,0,0,3,0,0,0,0,0,0,0
3,4,10.0,SPRING WIN,N Callan,P F Yiu,122.0,1076.0,2.0,2.00,7040.0,...,0,0,1,0,0,0,0,0,0,0
4,5,8.0,HAPPY COOPERATION,C Murray,T P Yung,124.0,1182.0,3.0,4.00,7068.0,...,0,0,2,0,0,0,0,0,0,0
5,6,9.0,LONDON MASTER,U Rispoli,D E Ferraris,123.0,1079.0,7.0,4.00,7070.0,...,0,1,1,0,0,0,0,0,0,0
6,7,2.0,VERY RICH MAN,M F Poon,L Ho,130.0,1077.0,1.0,5.00,7073.0,...,0,0,1,0,0,0,0,0,0,0
7,8,11.0,FIRE AND GOLD,A Badel,C H Yip,118.0,1071.0,9.0,5.00,7075.0,...,0,0,0,0,0,0,0,0,0,0
8,9,5.0,JUNZI,S de Sousa,W Y So,125.0,1015.0,8.0,5.00,7079.0,...,0,1,1,0,0,0,0,0,0,0
9,10,12.0,OPEN HOUSE,C Y Ho,D J Hall,115.0,1025.0,6.0,5.00,7084.0,...,0,0,0,0,0,0,0,0,0,0
